In [23]:
import json as json
import pathlib as pl
from copy import deepcopy
from datetime import datetime
from gc import collect
from os import path, system

import matplotlib.pyplot as plt
from matplotlib import rcParams
from numpy import array
from numpy.linalg import norm
from tqdm import tqdm

from calc_catenary_from_ext_points import get_coor_marker_points_ideal_catenary
from utils import check, gif_from_pngs, print_dict

gain = 1.5
rcParams[ 'figure.figsize' ] = [ gain * 6.4, gain * 4.8 / 2 ]

In [55]:
for v in pl.Path( "./export" ).glob( '*' ):
	print( str( v ).split( '//' )[ -1 ], end = '\t' )
	print( datetime.fromtimestamp( path.getctime( v ) ) )

export\bluerov_1726514489	2024-09-16 21:21:29.375715
export\three_robots_chain_1725023279	2024-08-30 15:07:59.322093
export\three_robots_chain_1725026180	2024-08-30 15:56:20.535888
export\three_robots_chain_1725027577	2024-08-30 16:19:37.793898
export\three_robots_chain_1725028953	2024-08-30 16:42:33.378778
export\three_robots_chain_1725033548	2024-08-30 17:59:08.196554
export\three_robots_chain_1725033930	2024-08-30 18:05:30.159590
export\three_robots_chain_1725264976	2024-09-12 22:05:01.853236
export\three_robots_chain_1725268112	2024-09-12 22:05:01.928170
export\three_robots_chain_1725274451	2024-09-12 22:05:02.010784
export\three_robots_chain_1725278516	2024-09-12 22:05:02.088836
export\three_robots_chain_1725525073	2024-09-12 22:05:02.166874
export\three_robots_chain_1725534896	2024-09-12 22:05:02.353150
export\three_robots_chain_1726033948	2024-09-11 07:52:28.967991
export\three_robots_chain_1726033954	2024-09-12 22:05:02.508750
export\three_robots_chain_1726033959	2024-09-12 22:

In [56]:
folder = pl.Path( './export/three_robots_chain_with_fixed_end_1727203158' )
uuid = str( folder ).split( '_' )[ -1 ]

In [57]:
with open( folder / 'config.json' ) as f:
	config = json.load( f )
print_dict( config )

predict: _predict_non_linear
get_actuation: _get_actuation_from_actual
get_result: _get_result_from_actual
model:
	dynamics: dynamics
	step: step
	model_dynamics:
		state_size: 48
		actuation_size: 9
		mass: 11.5
		center_of_mass: [0.0, 0.0, 0.0]
		weight: [0.0, 0.0, 112.77647499999999]
		center_of_volume: [0.0, 0.0, -0.02]
		buoyancy: [0.0, 0.0, -120.0]
		inverse_inertial_matrix: (6, 6)
		hydrodynamic_matrix: (6, 6)
		br0_pose: 0:6:None
		br0_position: 0:3:None
		br0_xy: 0:2:None
		br0_z: 2
		br0_orientation: 3:6:None
		br1_pose: 6:12:None
		br1_position: 6:9:None
		br1_xy: 6:8:None
		br1_z: 8
		br1_orientation: 9:12:None
		br2_pose: 12:18:None
		br2_position: 12:15:None
		br2_xy: 12:14:None
		br2_z: 14
		br2_orientation: 15:18:None
		br0_speed: 24:30:None
		br0_linear_speed: 24:27:None
		br0_angular_speed: 27:30:None
		br1_speed: 30:36:None
		br1_linear_speed: 30:33:None
		br1_angular_speed: 33:36:None
		br2_speed: 36:42:None
		br2_linear_speed: 36:39:None
		br2_angular_speed: 39:42:

In [58]:
slice_repository = { }
axes = [ 'x', 'y', 'z', 'phi', 'theta', 'psi', 'u', 'v', 'w', 'p', 'q', 'r' ]

for k, v in config[ 'model' ][ 'model_dynamics' ].items():
	if isinstance( v, str ) and ':' in v:
		slice_repository[ k ] = slice( *[ int( v ) if v != 'None' else None for v in v.split( ':' ) ] )

positions = { k: v for k, v in slice_repository.items() if 'position' in k }
orientations = { k: v for k, v in slice_repository.items() if 'orientation' in k }
positions_pairs = { a.split( '_' )[ 0 ]: b.split( '_' )[ 0 ] for a, b in
										zip( list( positions )[ :-1 ], list( positions )[ 1: ] ) }
linear_actuation = { k: v for k, v in slice_repository.items() if 'linear_actuation' in k }

In [59]:
for i, c in enumerate( config[ 'constraints' ] ):
	print( f'Constraint {i}' )
	for j, l in enumerate( c[ 'labels' ] ):
		print( f'\t{j}: {l}' )

Constraint 0
	0: $z_0+H_{01}$
	1: $z_1+H_{12}$
	2: $z_2+H_{2fe}$
	3: $|P_0^{x,y}-P_1^{x,y}|$
	4: $|P_1^{x,y}-P_2^{x,y}|$
	5: $|P_2^{x,y}-P_fe^{x,y}|$
	6: $|P_0^{x,y,z}-P_1^{x,y,z}|$
	7: $|P_1^{x,y,z}-P_2^{x,y,z}|$
	8: $|P_2^{x,y,z}-P_fe^{x,y,z}|$
	9: $|V_0|$
	10: $|V_1|$
	11: $|V_2|$


In [60]:
data_files = list( folder.glob( 'data/*' ) )
data_files.sort( key = lambda x: path.getmtime( x ) )
with open( data_files[ -1 ] ) as f:
	final_state = json.load( f )

times = [ config[ 'model' ][ 'time_step' ] * i for i in range( len( config[ 'target_trajectory' ] ) ) ]
target_trajectory = array( config[ 'target_trajectory' ] )[ :, 0 ]

n_frames = len( data_files )
previous_times = [ config[ 'model' ][ 'time_step' ] * i for i in range( n_frames ) ]
previous_target_trajectory = target_trajectory[ :n_frames ]
previous_actual_trajectory = array( final_state[ 'model' ][ 'previous_states' ] )[ 1:,
														 :config[ 'model' ][ 'model_dynamics' ][ 'state_size' ] // 2 ]

previous_actuations = array( final_state[ 'model' ][ 'previous_actuations' ] )

pose_weight = array( config[ 'pose_weight_matrix' ][ 0 ] )

In [61]:
check( f'{folder}/plots' )

0

In [62]:
for k, v in positions.items():
	if norm( pose_weight[ v, v ] ) == 0:
		continue
	absolute_distances = abs( previous_target_trajectory[ :, v ] - previous_actual_trajectory[ :, v ] )
	plt.plot( previous_times, absolute_distances )
	plt.legend(
			[
					r'over $\mathbf{x}_w$-axis',
					r'over $\mathbf{y}_w$-axis',
					r'over $\mathbf{z}_w$-axis'
					]
			)
	plt.xlabel( 'time [s]' )
	plt.ylabel( 'absolute error [m]' )
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/{k}_tracking_error_{uuid}.png', dpi = 300 )
	plt.close( 'all' )

In [63]:
for k, v in orientations.items():
	if norm( pose_weight[ v, v ] ) == 0:
		continue
	plt.plot( previous_times, abs( previous_target_trajectory[ :, v ] - previous_actual_trajectory[ :, v ] ) )
	plt.legend(
			[
					r'around $\mathbf{x}_w$-axis',
					r'around $\mathbf{y}_w$-axis',
					r'around $\mathbf{z}_w$-axis'
					]
			)
	plt.xlabel( 'time [s]' )
	plt.ylabel( 'absolute error [rad]' )
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/{k}_tracking_error_{uuid}.png', dpi = 300 )
	plt.close( 'all' )

In [64]:
for k, v in linear_actuation.items():
	plt.plot( previous_times, previous_actuations[ :-1, v ] )
	plt.xlabel( 'time [s]' )
	plt.ylabel( 'actuation [N]' )
	plt.legend(
			[
					r'over $\mathbf{x}_w$-axis',
					r'over $\mathbf{y}_w$-axis',
					r'over $\mathbf{z}_w$-axis'
					]
			)
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/{k}_{uuid}.png', dpi = 300 )
	plt.close( 'all' )

In [65]:
for p1, p2 in positions_pairs.items():
	absolute_distances = norm(
			previous_actual_trajectory[ :, positions[ p1 + '_position' ] ] -
			previous_actual_trajectory[ :, positions[ p2 + '_position' ] ],
			axis = 1
			)
	plt.plot( previous_times, absolute_distances )
plt.legend( [ f'from {a} to {b}' for a, b in positions_pairs.items() ] )
plt.xlabel( 'time [s]' )
plt.ylabel( 'distance [m]' )
plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
plt.savefig( f'{folder}/plots/distance_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [66]:
for p1, p2 in positions_pairs.items():
	absolute_distances = norm(
			previous_actual_trajectory[ :, slice_repository[ p1 + '_xy' ] ] -
			previous_actual_trajectory[ :, slice_repository[ p2 + '_xy' ] ],
			axis = 1
			)
	plt.plot( previous_times, absolute_distances )
plt.legend( [ f'from {a} to {b}' for a, b in positions_pairs.items() ] )
plt.xlabel( 'time [s]' )
plt.ylabel( 'distance [m]' )
plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
plt.savefig( f'{folder}/plots/horizontal_distance_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [67]:
figs = [ 'x', 'y', 'z' ]

x = plt.figure().subplots()
y = plt.figure().subplots()
z = plt.figure().subplots()
z.invert_yaxis()

legends = [ ]

for k, v in positions.items():
	position = previous_actual_trajectory[ :, v ]
	X = position[ :, 0 ]
	Y = position[ :, 1 ]
	Z = position[ :, 2 ]
	x.plot( previous_times, X )
	y.plot( previous_times, Y )
	z.plot( previous_times, Z )
	legends += [ f'{k}' ]
	if norm( pose_weight[ v, v ] ) > 0:
		target = previous_target_trajectory[ :, v ]
		Xt = target[ :, 0 ]
		Yt = target[ :, 1 ]
		Zt = target[ :, 2 ]
		x.plot( previous_times, Xt, ':', linewidth = 3 )
		y.plot( previous_times, Yt, ':', linewidth = 3 )
		z.plot( previous_times, Zt, ':', linewidth = 3 )
		legends += [ f'{k} target' ]

for f in plt.get_fignums():
	fig = plt.figure( f )
	plt.legend( legends )
	plt.xlabel( 'time [s]' )
	plt.ylabel( f'position on $\\mathbf{{{figs[ f - 1 ]}}}_w$-axis [m]' )
	plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
	plt.savefig( f'{folder}/plots/positions_on_{figs[ f - 1 ]}_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [68]:
plt.plot( previous_times, final_state[ 'compute_times' ] )
plt.hlines( config[ 'model' ][ 'time_step' ] * config[ 'horizon' ], 0, previous_times[ -1 ], 'r', '--' )
plt.legend( [ 'compute time', 'horizon depth' ] )
plt.xlabel( 'time [s]' )
plt.ylabel( 'time [s]' )
plt.subplots_adjust( bottom = 0.125, top = 1, left = 0.075, right = 1 )
plt.savefig( f'{folder}/plots/compute_times_{uuid}.png', dpi = 300 )
plt.close( 'all' )

In [16]:
frames_already_done = check( f'{folder}/plots/3d_frames' )

In [17]:
for frame in tqdm( range( frames_already_done, n_frames ) ):
	# frame = 100

	with open( data_files[ frame ] ) as f:
		simulation_state = json.load( f )
	predicted_trajectories = array( simulation_state[ 'predicted_trajectories' ] )

	fig = plt.figure( figsize = (10, 10) )
	view = plt.subplot( projection = '3d' )
	view.set_xlabel( r"$\mathbf{x}_w$-axis" )
	view.set_ylabel( r"$\mathbf{y}_w$-axis" )
	view.set_zlabel( r"$\mathbf{z}_w$-axis" )

	inset_view_xz = view.inset_axes( [ .0, .0, .2, .2 ] )
	inset_view_xz.set_xlabel( r"$\mathbf{x}_w$-axis" )
	inset_view_xz.set_ylabel( r"$\mathbf{z}_w$-axis" )
	inset_view_xz.invert_yaxis()

	inset_view_yz = view.inset_axes( [ .8, .0, .2, .2 ] )
	inset_view_yz.set_xlabel( r"$\mathbf{y}_w$-axis" )
	inset_view_yz.set_ylabel( r"$\mathbf{z}_w$-axis" )
	inset_view_yz.invert_xaxis()
	inset_view_yz.invert_yaxis()

	inset_view_xy = view.inset_axes( [ .0, .8, .2, .2 ] )
	inset_view_xy.set_xlabel( r"$\mathbf{x}_w$-axis" )
	inset_view_xy.set_ylabel( r"$\mathbf{y}_w$-axis" )
	inset_view_xy.invert_yaxis()

	for k, v in positions.items():
		if norm( pose_weight[ v, v ] ) > 0:
			Xt = target_trajectory[ :, v ][ :, 0 ]
			Yt = target_trajectory[ :, v ][ :, 1 ]
			Zt = target_trajectory[ :, v ][ :, 2 ]

			view.plot( Xt, Yt, Zt, ':' )
			view.scatter( Xt[ frame + 1 ], Yt[ frame + 1 ], Zt[ frame + 1 ], marker = 'x' )
			inset_view_xz.plot( Xt, Zt, ':' )
			inset_view_xz.scatter( Xt[ frame + 1 ], Zt[ frame + 1 ], marker = 'x' )
			inset_view_yz.plot( Yt, Zt, ':' )
			inset_view_yz.scatter( Yt[ frame + 1 ], Zt[ frame + 1 ], marker = 'x' )
			inset_view_xy.plot( Xt, Yt, ':' )
			inset_view_xy.scatter( Xt[ frame + 1 ], Yt[ frame + 1 ], marker = 'x' )

		X = previous_actual_trajectory[ frame, v ][ 0 ]
		Y = previous_actual_trajectory[ frame, v ][ 1 ]
		Z = previous_actual_trajectory[ frame, v ][ 2 ]

		view.scatter( X, Y, Z )
		inset_view_xz.scatter( X, Z )
		inset_view_yz.scatter( Y, Z )
		inset_view_xy.scatter( X, Y )

		for trajectory in predicted_trajectories:
			X = trajectory[ :, 0, v ][ :, 0 ]
			Y = trajectory[ :, 0, v ][ :, 1 ]
			Z = trajectory[ :, 0, v ][ :, 2 ]

			view.plot( X, Y, Z, linewidth = .5 )
			inset_view_xz.plot( X, Z, linewidth = .5 )
			inset_view_yz.plot( Y, Z, linewidth = .5 )
			inset_view_xy.plot( X, Y, linewidth = .5 )

	for p1, p2 in positions_pairs.items():
		position1 = deepcopy( previous_actual_trajectory[ frame, positions[ p1 + '_position' ] ] )
		position2 = deepcopy( previous_actual_trajectory[ frame, positions[ p2 + '_position' ] ] )
		position1[ 1: ] *= -1
		position2[ 1: ] *= -1

		try:
			cat12, _, _, _ = get_coor_marker_points_ideal_catenary( *position1, *position2, 3., .05 )
			cat12[ :, 1: ] *= -1
		except:
			cat12 = array( [ position1, position2 ] )

		view.plot( cat12[ :, 0 ], cat12[ :, 1 ], cat12[ :, 2 ] )
		inset_view_xz.plot( cat12[ :, 0 ], cat12[ :, 2 ] )
		inset_view_yz.plot( cat12[ :, 1 ], cat12[ :, 2 ] )
		inset_view_xy.plot( cat12[ :, 0 ], cat12[ :, 1 ] )

	view.axis( 'equal' )
	inset_view_xy.axis( 'equal' )
	inset_view_xz.axis( 'equal' )
	inset_view_yz.axis( 'equal' )
	view.invert_yaxis()
	view.invert_zaxis()

	plt.savefig( f'{folder}/plots/3d_frames/{frame}.png', dpi = 100 )
	plt.close( 'all' )
	del fig, view, inset_view_xz, inset_view_yz, inset_view_xy

100%|██████████| 9/9 [00:19<00:00,  2.12s/it]


In [18]:
collect()

2118073

In [19]:
gif_from_pngs( f'{folder}/plots/3d_frames', duration = config['model']['time_step'] * 1000 )

In [20]:
collect()

1474

In [21]:
system( f'ffmpeg -y -i {folder}/plots/3d_frames/animation.gif {folder}/plots/animation.mp4' )

0